In [1]:
from typing import Dict, Optional, List
import numpy as np

class HMMState:
    def __init__(self, mean: List[np.ndarray], covariance: List[np.ndarray], transition: Dict["HMMState", float], label: Optional[int] = None, parent: Optional["HMMState"] = None):
        self.mean = mean
        """n_gaussians of mean vectors."""
        self.covariance = covariance
        """n_gaussians of diagonal of covariance matrix."""
        self.transition = transition
        """Transition probability to other HMMState instances."""
        self.label = label
        """The digit associated with the state. `None` if the state is the first state."""
        self.parent = parent
        """The state is the first state if the `parent` is `None`."""

    @classmethod
    def root(cls):
        """Creates a root HMMState with default parameters."""
        return cls(mean=[], covariance=[], transition={}, label=None)

    def __hash__(self) -> int:
        """Enables HMMState instances to be used as dictionary keys or in sets."""
        return id(self)

    def add_transition(self, state: "HMMState", probability: float):
        """Adds or updates a transition probability to another state."""
        self.transition[state] = probability

    def get_transition_prob(self, state: "HMMState") -> float:
        """Retrieves the transition probability to another state, if defined."""
        return self.transition.get(state, 0.0)

# Example of usage
if __name__ == "__main__":
    # Creating root state
    root_state = HMMState.root()
    
    # Creating another state with example data
    mean_example = [np.array([0.0, 1.0])]
    covariance_example = [np.array([1.0, 1.0])]
    state_example = HMMState(mean=mean_example, covariance=covariance_example, transition={}, label=1)
    
    # Adding a transition from root to example state
    root_state.add_transition(state_example, 0.5)

    print(root_state.get_transition_prob(state_example))  # Example of getting a transition probability


0.5


In [ ]:
from typing import List, Tuple, Dict
import numpy as np

class HMM:
    def __init__(self):
        self.states: List[HMMState] = []
        self.observations: List[np.ndarray] = []
        self.state_index: Dict[HMMState, int] = {}

    def add_state(self, state: HMMState):
        """Adds a state to the HMM."""
        self.states.append(state)
        self.state_index[state] = len(self.states) - 1

    def set_observations(self, observations: List[np.ndarray]):
        """Sets the sequence of observations for the HMM."""
        self.observations = observations

    def viterbi(self) -> Tuple[List[HMMState], float]:
        """Finds the most likely sequence of states for the given observations using the Viterbi algorithm."""
        n_states = len(self.states)
        n_observations = len(self.observations)

        if n_states == 0 or n_observations == 0:
            return [], 0.0

        # Initialize the probability matrix and backpointer matrix
        dp = np.zeros((n_states, n_observations))
        backpointer = np.zeros((n_states, n_observations), dtype=int)

        # Initialization step
        for s in range(n_states):
            dp[s, 0] = self.states[s].get_emission_prob(self.observations[0]) * (1.0 / n_states)

        # Recursion step
        for t in range(1, n_observations):
            for s in range(n_states):
                max_prob, max_state = max(
                    (dp[prev_state, t-1] * self.states[prev_state].get_transition_prob(self.states[s]) * self.states[s].get_emission_prob(self.observations[t]), prev_state)
                    for prev_state in range(n_states)
                )
                dp[s, t] = max_prob
                backpointer[s, t] = max_state

        # Termination step
        max_prob = dp[:, -1].max()
        last_state = dp[:, -1].argmax()

        # Path backtracking
        best_path = [last_state]
        for t in range(n_observations - 1, 0, -1):
            last_state = backpointer[last_state, t]
            best_path.insert(0, last_state)

        best_states_sequence = [self.states[i] for i in best_path]
        return best_states_sequence, max_prob

    def get_emission_prob(self, state: HMMState, observation: np.ndarray) -> float:
        """Placeholder for emission probability calculation.
           Should be implemented based on the specific emission probability distribution."""
        return 1.0

# Note: You need to integrate this HMM class with the HMMState class from the previous example.
# Specifically, you might need to adjust the get_emission_prob method in HMMState or implement a new method
# in the HMM class to calculate emission probabilities based on the Gaussian distribution parameters (mean and covariance).

